# 第9章 装饰器和闭包

## 9.1 装饰器基础知识

装饰器的基本性质：
- 装饰器是一个函数或其他可调用对象。
- 装饰器可以把被装饰的函数替换成别的函数。
- 装饰器在加载（导入）模块时立即执行。

In [1]:
def deco(func):
    def inner():
        print('running inner')
    return inner

@deco
def target():
    print('running target()')

In [2]:
target()

running inner


In [3]:
target

<function __main__.deco.<locals>.inner()>

## 9.2 闭包

闭包：延伸了作用域的函数，包括函数（f）主体中引用的非全局变量和局部变量。这些变量必须来自包含f的外部函数的局部作用域。

In [4]:
# 一个计算累计平均值的高阶函数，所有值存储在历史数列series中
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [5]:
avg = make_averager()
avg(10)

10.0

In [6]:
avg(11)

10.5

In [7]:
avg(15)

12.0

In [8]:
# avg的局部变量
avg.__code__.co_varnames

('new_value', 'total')

In [10]:
# avg的自由变量
avg.__code__.co_freevars

('series',)

## 9.3 nonlocal声明

nonlocal：把变量标记为自由变量，便于在函数中为变量赋予新值。

In [11]:
# 计算累计平均值，不保存所有历史
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

## 9.4 变量查找逻辑

- 如果是`global x`声明，则`x`来自模块全局作用域，并赋予那个作用域中`x`的值。
- 如果是`nonlocal x`声明，则`x`来自最近一个定义它的外层函数，并赋予那个函数中局部变量`x`的值。
- 如果`x`是参数，或者在函数主体中赋了值，那么`x`就是局部变量。
- 如果引用了`x`，但是没有赋值也不是参数，则需要遵循以下规则：
    - 在外层函数主体的局部作用域（非局部作用域）内查找`x`。
    - 如果在外层作用域类没有找到，则从模块全局作用域内读取。
    - 如果在模块全局作用域内没有找到，则从`__builtins__.__dict__`中读取。

## 9.5 实现一个简单的装饰器

装饰器的主要作用是在每次调用被装饰的函数时计时，把运行时间、传入的参数和调用的结果打印出来。

In [15]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        # 调用被装饰的函数
        result = func(*args, **kwargs)
        
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        
        return result
    return clocked

In [16]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [17]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.13659960s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000060s] factorial(1) -> 1
[0.00001260s] factorial(2) -> 2
[0.00001960s] factorial(3) -> 6
[0.00002570s] factorial(4) -> 24
[0.00003420s] factorial(5) -> 120
[0.00004170s] factorial(6) -> 720
6! = 720


## 9.6 标准库中的装饰器

### 9.6.1 使用`functools.cache`做备忘

`functools.cache`装饰器实现了备忘，能把耗时的函数得到的结果保存起来，避免传入相同的参数时重复计算。

In [21]:
import functools


@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [22]:
fibonacci(6)

[0.00000070s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00019480s] fibonacci(2) -> 1
[0.00000050s] fibonacci(3) -> 2
[0.00020520s] fibonacci(4) -> 3
[0.00000040s] fibonacci(5) -> 5
[0.00021450s] fibonacci(6) -> 8


8

上述的叠放装饰器相当于`fibonacci = functools.cache(clock(fibonacci)`

### 9.6.2 单分派泛化函数

`functools.singledispatch`装饰器可以把整体方案拆分成多个模块，甚至可以为第三方包中无法编辑的类型提供专门的函数，将普通函数变成了泛化函数的入口，即为单分派。如果根据多个参数选择专门的函数，则是多分派。

**需求：**

开发一个调试Web应用程序的工具，生成HTML，以显示不同类型的Python对象。需要满足如下功能：
1. 当参数为`str`时，内部的换行符替换为`<br/>\n`，不使用`<pre>`标签，使用`<p>`。
2. 当参数为`int`时，以十进制和十六进制显示数（bool除外）。
3. 当参数为`list`时，输出一个HTML列表，根据各项的类型进行格式化。
4. 当参数为`float`和`Decimal`时，正常输出值，外加分数形式。

In [23]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str: 
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [24]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [25]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [33]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br/>\n- a game</p>'

In [27]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [28]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [29]:
htmlize(True)

'<pre>True</pre>'

In [30]:
htmlize(fractions.Fraction(2, 3))

'<pre>2/3</pre>'

In [31]:
htmlize(2/3)

'<pre>0.6666666666666666 (2/3)</pre>'

In [32]:
htmlize(decimal.Decimal('0.02380952'))

'<pre>0.02380952 (1/42)</pre>'

## 9.7 参数化装饰器

接收其他参数的装饰器：创建一个装饰器工厂函数来接收那些参数，然后再返回一个装饰器，应用到被装饰的函数上。

### 9.7.1 参数化clock装饰器

In [34]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT): # clock是参数化装饰器的工厂函数
    def decorate(func):     # 真正的装饰器 
        def clocked(*_args):  # 包装被装饰的函数 
            t0 = time.perf_counter()
            _result = func(*_args)  # 调用被装饰的函数
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) 
            result = repr(_result)  
            print(fmt.format(**locals()))
            return _result

        return clocked 

    return decorate

In [35]:
@clock()
def snooze(seconds):
    time.sleep(seconds)


for i in range(3):
    snooze(.123)

[0.13203000s] snooze(0.123) -> None
[0.12454020s] snooze(0.123) -> None
[0.12390610s] snooze(0.123) -> None


In [36]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12821319999784464s
snooze: 0.12511039999662898s
snooze: 0.1259561000042595s


In [37]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.126s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s


### 9.7.2 基于类的clock装饰器

In [38]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:

    def __init__(self, fmt=DEFAULT_FMT):  
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

## 9.8 杂谈

- 动态作用域与词法作用域：
    - 动态作用域：根据调用函数的环境求解自由变量。
    - 如果函数使用自由变量，程序员必须知道函数的内部细节，才能建立正确运行所需的环境。
    - 词法作用域：根据定义函数的环境求解自由变量。
    - 在JavaScript中，特殊变量this既可以使用词法作用域，也可以使用动态作用域。

- Python装饰器和装饰器设计模式：
    - Python函数装饰器符合对装饰器模式的一般描述：动态地给一个对象添加一些额外的职责，就扩展功能而言，装饰器模式比子类化更灵活。
    - 在实现层面，Python装饰器与装饰器设计模式不同，装饰器函数相当于Decorator的具体子类，而装饰器返回的内部函数相当于装饰器实例。